In [12]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import hashlib
import fitz

In [13]:
%%script false

#SCRAPING PROCESS: Needs to be run only once

def extract_pdf_url(page_id):
    url = 'https://eparlib.nic.in' + page_id
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    view_links = soup.find_all('a', string='View/Open')
    return view_links[0].get('href')

def download_file(url, output_folder, downloaded_files):
    filename = os.path.join(output_folder, url.split("/")[-1])

    # Check if file already exists by name
    if filename in downloaded_files:
        print(f"{filename} already downloaded by name, skipping...")
        return

    # Proceed with download without size check
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)
    downloaded_files.add(filename)

output_path = r'C:\Users\vishe\Desktop\Indian Parliament'

# Store downloaded filenames for comparison
downloaded_files = set()

for i in range(220, -1, -1):
    offset = i * 20
    url = f'https://eparlib.nic.in/handle/123456789/7?offset={offset}'

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    view_links = soup.find_all('a', string='View...')
    extracted_links = [link.get('href') for link in view_links]

    for link in extracted_links:
        pdf_url = 'https://eparlib.nic.in' + extract_pdf_url(link)
        if pdf_url:
            download_file(pdf_url, output_path, downloaded_files)


Couldn't find program: 'false'


In [14]:
%%script false

#READING PDF PROCESS: Needs to be run only once

pdf_folder = r"C:\Users\vishe\Desktop\Indian Parliament"

# Get all PDF file names in the folder
pdf_files = os.listdir(pdf_folder)
num_files = len(pdf_files)
print(f"Number of PDF files: {num_files}")

#Select batch
selected_batch=1
    
# Lists to accumulate values
file_names = []
file_contents = []

# Iterate over each PDF file in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)

        # Use PyMuPDF to open and read the PDF
        with fitz.open(pdf_path) as pdf_doc:
            # Create a variable with the file name
            variable_name = os.path.splitext(filename)[0]  # Removing the .pdf extension
            variable_content = ""

            # Iterate through each page in the PDF
            for page_num in range(pdf_doc.page_count):
                page = pdf_doc[page_num]
                variable_content += page.get_text()

            # Append values to the lists
            file_names.append(variable_name)
            file_contents.append(variable_content)

            # Print for demonstration purposes
            print(f"File Name: {variable_name}")

# Create a DataFrame from the lists
df = pd.DataFrame({"file_name": file_names, "file_content": file_contents})

# Save the DataFrame to a CSV file
df.to_csv(f'C:\Users\vishe\Desktop\Indian Parliament\practice\output_file.csv', index=False)

# Display the DataFrame
df.head(1)


Couldn't find program: 'false'


In [16]:
# Count unique values in the 'file_name' column
count = df['file_name'].nunique()

print("no:", count)


no: 340


In [17]:
import re

df['content_eng'] = df['file_content'].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', str(x)))

# Display the updated DataFrame
df.head(1)


,file_name,file_content,content_eng
0,LSD_0103_VII_2011 (10),07.12.2011 \n1\nC O N T E N T S \n \nFifteenth...,07122011 1C O N T E N T S Fifteenth Series Vo...


In [18]:
df['text_eng'] = df['content_eng'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['text_eng'] = df['text_eng'].apply(lambda x: re.sub(r'[0-9]+', '', str(x)))

df = df.dropna()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vishe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
from wordcloud import WordCloud
from nltk.corpus import stopwords as nltk_stopwords
import nltk
from nltk import FreqDist
import matplotlib.pyplot as plt
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from numpy.random import seed
import random


# Download NLTK resources 
nltk.download('punkt')
seed(10)

# Sample 150 random indices from the DataFrame length
random_indices = random.sample(range(len(df)), 150)

# Select the text at those indices
selected_text = df['text_eng'].iloc[random_indices]

# Concatenate the selected text with spaces
all_text = ' '.join(selected_text)

# Tokenize the text into words
words = word_tokenize(all_text)

# Filter out punctuation and non-alphabetic tokens
words = [word.lower() for word in words if word.isalpha()]

# Count the occurrences of each word
word_counts = Counter(words)

# Get stopwords from NLTK
nltk_stopwords_set = set(nltk_stopwords.words('english'))

custom_stopwords = ['sir', 'hon', 'shri', 'per', 'cent', 'government', 'india', 'will', 'central', 'people','question','papers', 'one','laid','prime','see','hrs', 'minister','would', 'also', 'dated', 'singh', 'dr', 'shrimati', 'interruptions', 'bill', 'state', 'women', 'kumar', 'house', 'country', 'speaker', 'district', 'sabha', 'report', 'library', 'tamil', 'distt', 'report', 'cum', 'lok', 'sabha', 'manipur', 'placed', 'want', 'regarding','like','may','shri','castesshri','reservedsch', 'eastshri','lai','southshri','sch','introduce'
    'answer','sixth','seventh','eighth','vol','xxvii','madam','chairperson','cannot'] 

frequent_stopwords = word_counts.most_common(300)
frequent_stopwords = [word for word, count in word_counts.most_common(300)]

stopwords_all = nltk_stopwords_set | set(custom_stopwords) | set(frequent_stopwords)

# Remove specific words from the list
words_to_remove = ['education','production','bank','library','railways','railway','health','delhi','english','rural','finance','agriculture','development','foreign','women','land','tamil','industry','financial']  # Replace with the words you want to remove

stopwords_all = stopwords_all - set(words_to_remove)

print(stopwords_all)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vishe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'his', 'xv', 'because', 'an', 'very', 'weren', 'hindi', 'xsi', 'it', 'make', 'mm', 'you', 'below', 'hrs', "won't", 'si', 'all', 'cum', 'mustn', 'than', 'mx', 'bd', 'nx', 'see', 'ca', 'shan', 'government', 'do', 'did', 'haven', 'couldn', 'sc', 'll', 'her', 'has', 'sixth', "wasn't", 'ji', 'one', 'shouldn', 've', 'the', 'these', 'said', "isn't", 'working', 'in', 'have', 'ourselves', 'nj', 'z', 'speech', 'or', 'dr', 'bev', 'go', 'gazette', 'report', 'into', 'member', "you've", 'between', 'standing', 'vxbe', 'cv', 'vol', 'again', 'how', 'water', 'new', 'would', "you'd", "didn't", 'herself', 'lai', 'p', 'ic', 'c', "shouldn't", 'abd', "weren't", "don't", 'itself', "doesn't", 'ma', 'mightn', 'bvx', 'ministry', 's', 'chairperson', 'statement', 'bebe', 'over', 'u', 'e', 'vxi', 'lok', 'y', 'national', 'please', 'inx', 'x', 'were', "aren't", 'bec', "haven't", 'ci', 'cn', 'give', 'been', 'bill', 'su', 'sch', 'cbvx', 'which', 'beci', 'nadu', 'must', 'per', 'members', 'eastshri', 'am', 'going', 'tab

In [20]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
import re

# Function to train LDA model for a document
def train_lda_for_doc(doc_bow, dictionary):
    return LdaModel(corpus=doc_bow, num_topics=5, id2word=dictionary)

# Preprocess all documents to build the dictionary
for index, row in df.iterrows():
    print(f"Processing document {index}...")
    processed_doc = re.sub(r'[^\w\s]', '', str(row['text_eng']))
    processed_doc = re.sub(r'[0-9]+', '', processed_doc)
    processed_doc = processed_doc.lower().split()  # Convert to lowercase
    processed_doc = [word for word in processed_doc if (len(word) > 2) and (word not in stopwords_all)]
    dictionary = corpora.Dictionary([processed_doc])  # Create dictionary representation of the document
    doc_bow = [dictionary.doc2bow(processed_doc)]  # Create bag-of-words representation of the document
    lda_model = train_lda_for_doc(doc_bow, dictionary)  # Train LDA model for the document
    # Initialize column to save words in topic 0
    df.at[index, 'topic_0_words'] = ''
    # Print the topics
    print("Topics:")
    for idx, topic in lda_model.show_topics(num_topics=1, formatted=False):
        print(f"Topic: {topic}")
        df.at[index,'topic_0_words']=topic


Processing document 0...
Topics:
Topic: [('dvc', 0.004516206), ('power', 0.0037982047), ('valley', 0.0033419256), ('damodar', 0.0031789257), ('prices', 0.0028883757), ('corporation', 0.0024299175), ('growth', 0.0023544815), ('delhi', 0.0022340938), ('english', 0.0021420661), ('areas', 0.0021022263)]
Processing document 1...
Topics:
Topic: [('english', 0.014454378), ('notification', 0.011863507), ('library', 0.011624482), ('land', 0.011441645), ('operation', 0.01078753), ('highway', 0.010672736), ('building', 0.010629375), ('maintenance', 0.01023632), ('management', 0.010191753), ('acquisition', 0.008477791)]
Processing document 2...
Topics:
Topic: [('notification', 0.0063067423), ('bangladesh', 0.0060187983), ('english', 0.0054340637), ('companies', 0.004239211), ('courts', 0.0041663554), ('court', 0.0034909677), ('library', 0.003477693), ('rules', 0.00335145), ('beg', 0.0032040004), ('development', 0.0031925172)]
Processing document 3...
Topics:
Topic: [('english', 0.0115002375), ('cv

In [21]:
df.head(3)

,file_name,file_content,content_eng,text_eng,topic_0_words
0,LSD_0103_VII_2011 (10),07.12.2011 \n1\nC O N T E N T S \n \nFifteenth...,07122011 1C O N T E N T S Fifteenth Series Vo...,C O N T E N T S Fifteenth Series VolXX Ninth...,"[(dvc, 0.004516206), (power, 0.0037982047), (v..."
1,LSD_0103_VII_2011 (11),08.08.2011 \n \n \n \n \n \n \n \n1\nC O N T E...,08082011 1C O N T E N T S Fifteenth Se...,C O N T E N T S Fifteenth Series Vol ...,"[(english, 0.014454378), (notification, 0.0118..."
2,LSD_0103_VII_2011 (12),"C O N T E N T S \n \nFifteenth Series, Vol. XI...",C O N T E N T S Fifteenth Series Vol XIX Eigh...,C O N T E N T S Fifteenth Series Vol XIX Eigh...,"[(notification, 0.0063067423), (bangladesh, 0...."


In [22]:
# Define a function to apply re.findall to each element
def extract_words(observation):
    return re.findall(r"\('(.*?)',", observation)

# Apply the function to the column after converting to string
df['words'] = df['topic_0_words'].apply(lambda x: str(x)).apply(extract_words)


In [23]:
df.head(3)

,file_name,file_content,content_eng,text_eng,topic_0_words,words
0,LSD_0103_VII_2011 (10),07.12.2011 \n1\nC O N T E N T S \n \nFifteenth...,07122011 1C O N T E N T S Fifteenth Series Vo...,C O N T E N T S Fifteenth Series VolXX Ninth...,"[(dvc, 0.004516206), (power, 0.0037982047), (v...","[dvc, power, valley, damodar, prices, corporat..."
1,LSD_0103_VII_2011 (11),08.08.2011 \n \n \n \n \n \n \n \n1\nC O N T E...,08082011 1C O N T E N T S Fifteenth Se...,C O N T E N T S Fifteenth Series Vol ...,"[(english, 0.014454378), (notification, 0.0118...","[english, notification, library, land, operati..."
2,LSD_0103_VII_2011 (12),"C O N T E N T S \n \nFifteenth Series, Vol. XI...",C O N T E N T S Fifteenth Series Vol XIX Eigh...,C O N T E N T S Fifteenth Series Vol XIX Eigh...,"[(notification, 0.0063067423), (bangladesh, 0....","[notification, bangladesh, english, companies,..."


In [25]:
df.to_csv(r'C:\Users\vishe\Desktop\Indian Parliament\missing\final_missing.csv', index=False)
